In [ ]:
import cv2
import math
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("jumping jacks 2.mp4")  # Capture video from webcam
jj_count = 0
jj_detected = False

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty camera frame.")
            continue

        # Convert the BGR image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect the pose landmarks.
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks is not None:
            # Get the landmarks of the left and right shoulder, elbow, wrist, and hip.
            left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
            right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
            left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
            right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
            left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
            right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
            left_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
            right_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]
            left_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
            right_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]

            # Calculate the angles of the left and right arms with respect to the body.
            left_arm_angle = (180 - math.degrees(math.atan2(left_wrist.y - left_elbow.y, left_wrist.x - left_elbow.x)
                                                 - math.atan2(left_elbow.y - left_shoulder.y, left_elbow.x - left_shoulder.x)))
            right_arm_angle = (180 - math.degrees(math.atan2(right_wrist.y - right_elbow.y, right_wrist.x - right_elbow.x)
                                                  - math.atan2(right_elbow.y - right_shoulder.y, right_elbow.x - right_shoulder.x)))

            # Calculate the angles of the left and right legs with respect to the body.
            left_leg_angle = (180 - math.degrees(math.atan2(left_knee.y - left_hip.y, left_knee.x - left_hip.x)
                                                 - math.atan2(left_ankle.y - left_knee.y, left_ankle.x - left_knee.x)))
            right_leg_angle = (180 - math.degrees(math.atan2(right_knee.y - right_hip.y, right_knee.x - right_hip.x)
                                                  - math.atan2(right_ankle.y - right_knee.y, right_ankle.x - right_knee.x)))

            # Check if the left and right arms are extended and the left and right legs are together.
            if left_arm_angle < 220 and right_arm_angle < 220 and abs(left_leg_angle - right_leg_angle) < 320:
                if not jj_detected:
                    jj_count += 1
                    jj_detected = True
            else:
                jj_detected = False

            # Draw the pose landmarks and the angles on the image.
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.putText(image, f"Left Arm Angle: {int(left_arm_angle)}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right Arm Angle: {int(right_arm_angle)}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f"Left Leg Angle: {int(left_leg_angle)}", (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right Leg Angle: {int(right_leg_angle)}", (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f"Jumping Jacks: {jj_count}", (20, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Jumping Jack Counter', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()